In [7]:
# -*- encoding:utf-8 -*-
import numpy as np
import pickle
import keras
from keras.utils import *
from keras.layers import *
from keras.models import *
from keras.callbacks import ModelCheckpoint
import argparse
from keras.preprocessing.sequence import pad_sequences
import os

Using TensorFlow backend.


In [1]:
import numpy as np

In [2]:
onehot = np.fromfile('onehot.txt',dtype=np.uint8)

In [3]:
base_shape = (6342, 68, 2000)
onehot = onehot.reshape(base_shape)#reshaping to base shape

In [4]:
#returning one hot for checking padding
def show(a):
    arr = []
    for item in a:
        arr.append(np.argmax(item))
    return arr

In [9]:
# the inputs should be already embedded ==> check prepare_dataset.py for further information
seq_length = 68
emb_dim = 2000
hidden_size = 8
batch_size = 32
n_epochs = 1
opt = 'adam'
embedded_inputs = Input(shape=(seq_length, emb_dim))
encoded_inputs = LSTM(hidden_size,return_sequences=True, name="encoder")(embedded_inputs)
decoded = RepeatVector(seq_length)(encoded_inputs)
decoded = LSTM(emb_dim, return_sequences=True)(decoded)     
autoencoder = Model(embedded_inputs, decoded)
autoencoder.compile(loss='mse', optimizer=opt) # we use 'mse' loss function to train our autoencoder

In [10]:
print(autoencoder.summary())
print("Now loading data...")    
print(onehot.shape)    
print('Found %s sequences.' % len(onehot))    
print("Now training the model...")    
# we automatically save the best autoencoder model during the training
checkpoint = ModelCheckpoint(filepath='model.h5', save_best_only=True)                
autoencoder.fit(onehot, onehot,
        batch_size=batch_size, epochs=n_epochs, verbose=1, validation_split=0.2, callbacks=[checkpoint])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 68, 2000)          0         
_________________________________________________________________
encoder (LSTM)               (None, 32)                260224    
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 68, 32)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 68, 2000)          16264000  
Total params: 16,524,224
Trainable params: 16,524,224
Non-trainable params: 0
_________________________________________________________________
None
Now loading data...
(6342, 68, 2000)
Found 6342 sequences.
Now training the model...
Train on 5073 samples, validate on 1269 samples
Epoch 1/1
5073/5073 [==============================] - 1082s 213ms/step - loss: 2.5864e-04 - val_loss: 1.7129e-04


In [18]:
show(autoencoder.predict(onehot[0:1])[0])

[1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999,
 1999]

In [ ]:
#save model shape
model_json = autoencoder.to_json()
with open("autoencoder.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model_complete.h5")

In [ ]:
#load mode
json_file = open('autoencoder.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
autoencoder = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model_complete.h5")